# Diurnal Breeze Identification Method (BreezeID)

This script is designed to detect land and sea breeze events using wind speed and direction data. The data is first transformed into a shore-parallel coordinate system, then filtered and analyzed to identify breeze events based on specific criteria. The script outputs two tables summarizing the characteristics of the detected land and sea breeze events. This script is designed to handle a single NetCDF file as the data import file. A separate script, titled "BreezeID_csv," has been developed to handle CSV files. The script is designed such that only the user-specified parameters need to be changed. A step-by-step guide outlining each of the user-specified parameters is provided in the README file. 

## Outline of Script Workflow 

1. User Defined Parameters
    * Time Definitions, Detection Windows, Data Range, File Locations, Coastline Shape, Filtering Parameters, Output Parameters
<br>
<br>
2. Library Calls
    * Loading necessary libraries
<br>
<br>
3. Function Definitions
    * Various functions for calculations, data filtering, and breeze detection
<br>
<br>
4. Variable Creation
    * Time variables, day vectors, filtering MJO data
<br>
<br>
5. Data Import
    * Reading and combining data files
<br>
<br>
6. Grid Rotation
    * Calculating the rotated u and v components of wind
<br>
<br>
7. Filtering Algorithm
    * Isolating the diurnal part of the flow
<br>
<br>
8. Detection Algorithm Execution
    * Running the event identification criteria
<br>
<br>
9. Data Table Output
    * Creating and exporting output tables

## User-Specified Parameters

In [ ]:
##### Time Definitions #####

# UTC to Local Time #
local_zero_utc = 7                              # Specify what 0 UTC is in local 24-hour time
sampling_rate = 1                               # Specify the sampling rate in terms of hours (e.g, 1 for hourly, 0.5 for half-hourly)

# Land Breeze Detection 24-Hour Window #
land_breeze_window_start_time = 10              # Specify when to start a new land breeze day in local 24-hour time

# Sea Breeze Detection 24-Hour Window #
sea_breeze_window_start_time = 7                # Specify when to start a new sea breeze day in local 24-hour time

# Data Range #
data_start_date = "2018-01-01"                  # The start date of your data in YYYY-MM-DD
data_end_date = "2018-12-31"                    # The end date of your data in YYYY-MM-DD

##### MJO RMM File #####
MJO_RMM_file_location = "path/to/modified_rmm.74toRealtime.txt" # Path to folder with MJO information

##### Data Files #####
data_file_path = "path/to/data_directory"       # Path to folder with data files
file_type = "nc"                                # Currently designed to handle NetCDF files
u_comp_name = "u10"                             # User-defined variable name for u component wind
v_comp_name = "v10"                             # User-defined variable name for v component wind

lat_interest = -4.0                             # Latitude of analysis coordinate
lon_interest = 102.25                           # Longitude of analysis coordinate 

#####Coastline Shape#####

# User-specified rotation angle
coordinate_axis_rotation_user = 40.00            # Change to FALSE if using calculated angle

# # Latitude and Longitude for three points
# lat1 = -4.26  # Point 1 Latitude
# lon1 = 102.69 # Point 1 Longitude
# lat2 = -3.86  # Point 2 Latitude
# lon2 = 102.30 # Point 2 Longitude
# lat3 = -3.52  # Point 3 Latitude
# lon3 = 102.02 # Point 3 Longitude

#####Filtering Parameters#####
synoptic_filter_period = 3                      # Defined in days
noise_filter_period = 7                         # Defined in hours

#####Data Table Output Parameters#####
data_table_output_directory = "path/to/data_output_directory"       # File path to desired output data directory
output_file_name_land = "land_breeze_output_table.csv"              # File name for land breeze output table (CSV)
output_file_name_sea  = "sea__breeze_output_table.csv"              # File name for sea breeze output table (CVS)

## Libraries

In [ ]:
library(ncdf4)     # Provides tools for reading, writing, and manipulating NetCDF files
library(dplR)      # Contains functions for filtering and analyzing data
library(lubridate) # Facilitates working with dates and times by providing functions for parsing, manipulating, and performing arithmetic on date-time objects
library(dplyr)     # Offers a set of tools for data manipulation, including functions for filtering, selecting, and transforming data frames
library(zoo)       # Provides infrastructure for working with regular and irregular time series data, including functions for creating, indexing, and merging time series objects

## Functions

### General Functions

In [ ]:
######Calculate the number of days#####
calculate_num_days = function(start_date, end_date) {
  num_days = as.numeric(difftime(ymd(end_date), ymd(start_date), units = "days")) + 1
  return(num_days)
}

#####Create the time vector#####
create_time_vector = function(local_0_utc, num_days, sampling_rate) {
  if (local_0_utc == 0) {
    # Define the time sequence for one day starting from midnight
    time_seq = seq(0, 23, by = sampling_rate)
  } else {
    # Define the time sequence for one day based on the specified local time for 0 UTC
    time_seq = c(seq(local_0_utc, 23, by = sampling_rate), seq(0, local_0_utc - sampling_rate, by = sampling_rate))
  }
  # Replicate the time sequence for the specified number of days
  timefile = rep(time_seq, num_days)
  return(timefile)
}

#####Create the days of year table#####
create_days_of_year_table = function(start_date, end_date) {
  # Generate date range
  date_range = seq(ymd(start_date), ymd(end_date), by = "days")
  
  # Create data frame
  df = data.frame(
    Day_num = 1:length(date_range),
    Date = format(date_range, "%d-%b-%Y"),
    Year = year(date_range),
    Month = month(date_range),
    Day_of_Month = day(date_range)
  )
  
  return(df)
}

#####Filter MJO data by date range####
filter_mjo_data = function(file_path, start_date, end_date) {
  # Read the MJO data file
  MJO_file = read.table(file_path, header = TRUE)
  
  # Combine year, month, and day columns to create a date column
  MJO_file = MJO_file %>%
    mutate(Date = make_date(year, month, day))
  
  # Filter the data based on the specified date range
  filtered_mjo = MJO_file %>%
    filter(Date >= as.Date(start_date) & Date <= as.Date(end_date))
  
  return(filtered_mjo)
}

#####Land/Sea Breeze Day Vector Definition#####
generate_day_vector= function(local_time_at_0_utc, start_counting_hour, sampling_frequency, num_days) {
  
  crossing_midnight = FALSE
  
  repeat {
    # Adjust start_counting_hour if crossing midnight
    if (crossing_midnight) {
      start_counting_hour = start_counting_hour + 24
    }
    
    # Calculate the number of observations before the start counting hour
    observations_before_start = (start_counting_hour - local_time_at_0_utc) %% 24 / sampling_frequency
    
    # Initialize the day vector
    day_vector = c()
    
    if (local_time_at_0_utc < start_counting_hour) {
      # Use 0s for the observations before the start counting hour
      day_vector = rep(0, observations_before_start)
      day_vector = c(day_vector, rep(1, 24 / sampling_frequency))
    } else if (local_time_at_0_utc == start_counting_hour) {
      # Handle the case where local_time_at_0_utc is equal to start_counting_hour
      day_vector = rep(1, 24 / sampling_frequency)
    } else {
      # Start counting from 1 with fewer observations for the first day
      observations_for_first_day = 24 - observations_before_start
      day_vector = rep(1, 24 - observations_for_first_day)
    }
    
    # Calculate the number of observations per day
    observations_per_day = 24 / sampling_frequency
    
    # Loop through each day of the year starting from the second day
    for (i in 2:num_days) {
      day_vector = c(day_vector, rep(i, observations_per_day))
    }
    
    # Remove the last few elements to ensure the vector length matches the expected length
    expected_length = num_days * observations_per_day
    day_vector = day_vector[1:expected_length]
    
    # Check for NA values
    if (any(is.na(day_vector))) {
      crossing_midnight = TRUE
    } else {
      break
    }
  }
  
  return(day_vector)
}

#####Zero-Crossing Calculation Function#####
zero_crossing=function(time_1,value_t1,time_2,value_t2){
  zero_value=(time_1*value_t2-time_2*value_t1)/(value_t2-value_t1)
  return(zero_value)
}

#####Index to Time Function#####
convert_index_to_time = function(index, time_file, sampling_frequency) {
  # Get the integer part and the fractional part of the index
  int_part = floor(index)
  frac_part = index - int_part
  
  # Get the time at the integer part of the index
  time_at_int = time_file[int_part]
  
  # Calculate the interpolated time
  interpolated_time = time_at_int + frac_part * sampling_frequency
  
  # Adjust for wrap-around at 24 hours
  if (interpolated_time >= 24) {
    interpolated_time = interpolated_time - 24
  }
  
  return(interpolated_time)
}

#####Rotation Angle Calculation#####
calculate_angle_from_x_axis = function(lat1, lon1, lat2, lon2, lat3, lon3) {
  # Combine points into a data frame
  points = data.frame(
    lat = c(lat1, lat2, lat3),
    lon = c(lon1, lon2, lon3)
  )
  
  # Fit a linear model
  fit = lm(lat ~ lon, data = points)
  
  # Extract the slope of the best fit line
  slope = coef(fit)[2]
  
  # Calculate the angle from the positive x-axis
  angle = atan(slope) * 180 / pi
  
  # Ensure the angle is defined positive for counterclockwise rotation
  if (angle < 0) {
    angle = angle + 360
  }
  
  # Angle of rotation to make the positive x-axis perpendicular to the best fit line
  angle = angle + 90
  
  # Ensure the angle is within the range of 0 to 360 degrees
  if (angle > 360) {
    angle = angle - 360
  }
  
  return(as.numeric(angle))
}

### Data Output Functions

In [ ]:
#####Table Creator Functions: Land#####
table_maker_land = function(data_matrix, local_0UTC_time, land_breeze_window_start_time){
  event = c("Classic", "Interruption")
  end = dim(data_matrix)[1]
  z = seq(1, end, by = 1)
  output_matrix = matrix(nrow = length(z), ncol = 14)
  icount = 1
  first_go_around = TRUE
  date_tracker = list()
  
  for (num in z){
    if (is.na(data_matrix[num, 27]) == TRUE & is.na(data_matrix[num, 37]) == TRUE){
    }
    if (is.na(data_matrix[num, 32]) == FALSE){
      
      # Check the day vector #
      if (first_go_around) {
        if (day_vector_land[num] == 0) {
          day_vector_land = day_vector_land + 1
          zero_utc_condition = TRUE
          first_go_around = FALSE
          
        } else {
          day_vector_land = day_vector_land
          zero_utc_condition = FALSE
          first_go_around = FALSE
        }
      }
      
      ####Interruption Events####
      
      day_index = which(days_of_year_table[, 1] == day_vector_land[num])
      date = paste(days_of_year_table[day_index, 4], days_of_year_table[day_index, 5], days_of_year_table[day_index, 3], sep = "-")
      
      if (date %in% date_tracker) {
        
        # Check the second duplicate onset time value
        if (as.numeric(data_matrix[num, 32]) < land_breeze_window_start_time) {
          output_matrix[icount, 7] = days_of_year_table[day_index, 5] + 1
          next_day_index = day_index + 1
          output_matrix[icount, 8] = mjo_data[next_day_index, 6]
          output_matrix[icount, 9] = round(mjo_data[next_day_index, 7], digits = 2)
          output_matrix[icount, 14] = days_of_year_table[next_day_index, 3]
          output_matrix[icount - 1, 14] = days_of_year_table[day_index, 3]
          
        } else {
          output_matrix[icount, 7] = days_of_year_table[day_index, 5]
          output_matrix[icount, 8] = mjo_data[day_index, 6]
          output_matrix[icount, 9] = round(mjo_data[day_index, 7], digits = 2)
        }
      } else {
        date_tracker = c(date_tracker, date)
        output_matrix[icount, 14] = days_of_year_table[day_index, 3]
        output_matrix[icount, 7] = days_of_year_table[day_index, 5]
        output_matrix[icount, 8] = mjo_data[day_index, 6]
        output_matrix[icount, 9] = round(mjo_data[day_index, 7], digits = 2)
      }
      
      output_matrix[icount, 1] = days_of_year_table[day_index, 4]
      output_matrix[icount, 2] = data_matrix[num, 32]
      output_matrix[icount, 3] = data_matrix[num, 22]
      output_matrix[icount, 4] = data_matrix[num, 36]
      output_matrix[icount, 5] = abs(round(data_matrix[num, 35], digits = 2))
      output_matrix[icount, 11] = data_matrix[num, 28]
      output_matrix[icount, 12] = data_matrix[num, 30]
      output_matrix[icount, 13] = data_matrix[num, 12]
      output_matrix[icount, 6] = abs(round(data_matrix[num, 10], digits = 2))
      
      ####Event Type and Iteration####
      output_matrix[icount, 10] = event[2]
      icount = icount + 1
    }
    if (is.na(data_matrix[num, 27]) == FALSE){
      
      # Check the day vector #
      if (first_go_around) {
        if (day_vector_land[num] == 0) {
          day_vector_land = day_vector_land + 1
          zero_utc_condition = TRUE
          first_go_around = FALSE
          
        } else {
          day_vector_land = day_vector_land
          zero_utc_condition = FALSE
          first_go_around = FALSE
        }
      }
      
      ####Classic Events####
      day_index = which(days_of_year_table[, 1] == day_vector_land[num])
      date = paste(days_of_year_table[day_index, 4], days_of_year_table[day_index, 5], days_of_year_table[day_index, 3], sep = "-")
      
      if (date %in% date_tracker) {
        
        # Check the second duplicate onset time value
        if (as.numeric(data_matrix[num, 27]) < land_breeze_window_start_time) {
          output_matrix[icount, 7] = days_of_year_table[day_index, 5] + 1
          next_day_index = day_index + 1
          output_matrix[icount, 8] = mjo_data[next_day_index, 6]
          output_matrix[icount, 9] = round(mjo_data[next_day_index, 7], digits = 2)
          output_matrix[icount, 14] = days_of_year_table[next_day_index, 3]
          output_matrix[icount - 1, 14] = days_of_year_table[day_index, 3]
          
        } else {
          output_matrix[icount, 7] = days_of_year_table[day_index, 5]
          output_matrix[icount, 8] = mjo_data[day_index, 6]
          output_matrix[icount, 9] = round(mjo_data[day_index, 7], digits = 2)
        }
      } else {
        date_tracker = c(date_tracker, date)
        output_matrix[icount, 14] = days_of_year_table[day_index, 3]
        output_matrix[icount, 7] = days_of_year_table[day_index, 5]
        output_matrix[icount, 8] = mjo_data[day_index, 6]
        output_matrix[icount, 9] = round(mjo_data[day_index, 7], digits = 2)
      }
      
      output_matrix[icount, 1] = days_of_year_table[day_index, 4]
      output_matrix[icount, 2] = data_matrix[num, 27]
      output_matrix[icount, 3] = data_matrix[num, 16]
      output_matrix[icount, 4] = data_matrix[num, 18]
      output_matrix[icount, 5] = abs(round(data_matrix[num, 17], digits = 2))
      output_matrix[icount, 11] = 9999
      output_matrix[icount, 12] = 9999
      output_matrix[icount, 13] = 9999
      output_matrix[icount, 6] = 9999
      
      ####Event Type and Iteration####
      output_matrix[icount, 10] = event[1]
      icount = icount + 1
    }
  }
  
  # New conditional statement for duplicate dates with equal onset time and duration
  unique_dates = unique(output_matrix[, c(1, 2, 3)])
  for (i in 1:nrow(unique_dates)) {
    duplicate_rows = which(output_matrix[, 1] == unique_dates[i, 1] & output_matrix[, 2] == unique_dates[i, 2] & output_matrix[, 3] == unique_dates[i, 3])
    if (length(duplicate_rows) > 1) {
      output_matrix[duplicate_rows[1], 14] = days_of_year_table[which(days_of_year_table[, 1] == unique_dates[i, 1]), 3]
      output_matrix = output_matrix[-duplicate_rows[-1], ]
    }
  }
  
  large_table = data.frame("Year" = as.numeric(output_matrix[, 14]),
                           "Month" = as.numeric(output_matrix[, 1]),
                           "Day" = as.numeric(output_matrix[, 7]),
                           "Onset_time" = round(as.numeric(output_matrix[, 2]), digits = 2),
                           "Duration" = round(as.numeric(output_matrix[, 3]), digits = 2),
                           "Time_Max_Speed" = as.numeric(output_matrix[, 4]),
                           "Max_Speed" = as.numeric(output_matrix[, 5]),
                           "MJO_Phase" = as.numeric(output_matrix[, 8]),
                           "MJO_Mag" = as.numeric(output_matrix[, 9]),
                           "Event_Type" = output_matrix[, 10],
                           "Flip2_Onset_Time" = round(as.numeric(output_matrix[, 11]), digits = 2),
                           "Flip2_Duration" = round(as.numeric(output_matrix[, 12]), digits = 2),
                           "Flip2_Time_Max_speed" = as.numeric(output_matrix[, 13]),
                           "Flip2_Max_Speed" = as.numeric(output_matrix[, 6]))
  
  return(na.omit(large_table))
}

#####Table Creator Function: Sea#####
table_maker_sea = function(data_matrix, local_0UTC_time, sea_breeze_window_start_time){
  event = c("Classic", "Interruption")
  end = dim(data_matrix)[1]
  z = seq(1, end, by = 1)
  output_matrix = matrix(nrow = length(z), ncol = 14)
  icount = 1
  first_go_around = TRUE
  date_tracker = list()
  
  for (num in z){
    if (is.na(data_matrix[num, 25]) == TRUE & is.na(data_matrix[num, 37]) == TRUE){
    }
    if (is.na(data_matrix[num, 31]) == FALSE){
      
      # Check the day vector #
      if (first_go_around) {
        if (day_vector_sea[num] == 0) {
          day_vector_sea = day_vector_sea + 1
          zero_utc_condition = TRUE
          first_go_around = FALSE
          
        } else {
          day_vector_sea = day_vector_sea
          zero_utc_condition = FALSE
          first_go_around = FALSE
        }
      }
      
      ####Interruption Event####
      
      day_index = which(days_of_year_table[, 1] == day_vector_sea[num])
      date = paste(days_of_year_table[day_index, 4], days_of_year_table[day_index, 5], days_of_year_table[day_index, 3], sep = "-")
      
      if (date %in% date_tracker) {
        
        # Check the second duplicate onset time value
        if (as.numeric(data_matrix[num, 31]) < sea_breeze_window_start_time) {
          output_matrix[icount, 7] = days_of_year_table[day_index, 5] + 1
          next_day_index = day_index + 1
          output_matrix[icount, 8] = mjo_data[next_day_index, 6]
          output_matrix[icount, 9] = round(mjo_data[next_day_index, 7], digits = 2)
          output_matrix[icount, 14] = days_of_year_table[next_day_index, 3]
          output_matrix[icount - 1, 14] = days_of_year_table[day_index, 3]
          
        } else {
          output_matrix[icount, 7] = days_of_year_table[day_index, 5]
          output_matrix[icount, 8] = mjo_data[day_index, 6]
          output_matrix[icount, 9] = round(mjo_data[day_index, 7], digits = 2)
        }
      } else {
        date_tracker = c(date_tracker, date)
        output_matrix[icount, 14] = days_of_year_table[day_index, 3]
        output_matrix[icount, 7] = days_of_year_table[day_index, 5]
        output_matrix[icount, 8] = mjo_data[day_index, 6]
        output_matrix[icount, 9] = round(mjo_data[day_index, 7], digits = 2)
      }
      
      output_matrix[icount, 1] = days_of_year_table[day_index, 4]
      output_matrix[icount, 2] = data_matrix[num, 31]
      output_matrix[icount, 3] = data_matrix[num, 19]
      output_matrix[icount, 4] = data_matrix[num, 34]
      output_matrix[icount, 5] = abs(round(data_matrix[num, 33], digits = 2))
      output_matrix[icount, 11] = data_matrix[num, 26]
      output_matrix[icount, 12] = data_matrix[num, 29]
      output_matrix[icount, 13] = data_matrix[num, 11]
      output_matrix[icount, 6] = abs(round(data_matrix[num, 8], digits = 2))
      
      ####Event Type and Iteration####
      output_matrix[icount, 10] = event[2]
      icount = icount + 1
    }
    if (is.na(data_matrix[num, 25]) == FALSE){
      
      # Check the day vector #
      if (first_go_around) {
        if (day_vector_sea[num] == 0) {
          day_vector_sea = day_vector_sea + 1
          zero_utc_condition = TRUE
          first_go_around = FALSE
          
        } else {
          day_vector_sea = day_vector_sea
          zero_utc_condition = FALSE
          first_go_around = FALSE
        }
      }
      
      day_index = which(days_of_year_table[, 1] == day_vector_sea[num])
      date = paste(days_of_year_table[day_index, 4], days_of_year_table[day_index, 5], days_of_year_table[day_index, 3], sep = "-")
      
      if (date %in% date_tracker) {
        
        # Check the second duplicate onset time value
        if (as.numeric(data_matrix[num, 25]) < sea_breeze_window_start_time) {
          output_matrix[icount, 7] = days_of_year_table[day_index, 5] + 1
          next_day_index = day_index + 1
          output_matrix[icount, 8] = mjo_data[next_day_index, 6]
          output_matrix[icount, 9] = round(mjo_data[next_day_index, 7], digits = 2)
          output_matrix[icount, 14] = days_of_year_table[next_day_index, 3]
          output_matrix[icount - 1, 14] = days_of_year_table[day_index, 3]
          
        } else {
          output_matrix[icount, 7] = days_of_year_table[day_index, 5]
          output_matrix[icount, 8] = mjo_data[day_index, 6]
          output_matrix[icount, 9] = round(mjo_data[day_index, 7], digits = 2)
        }
      } else {
        date_tracker = c(date_tracker, date)
        output_matrix[icount, 14] = days_of_year_table[day_index, 3]
        output_matrix[icount, 7] = days_of_year_table[day_index, 5]
        output_matrix[icount, 8] = mjo_data[day_index, 6]
        output_matrix[icount, 9] = round(mjo_data[day_index, 7], digits = 2)
      }
      
      output_matrix[icount, 1] = days_of_year_table[day_index, 4]
      output_matrix[icount, 2] = data_matrix[num, 25]
      output_matrix[icount, 3] = data_matrix[num, 13]
      output_matrix[icount, 4] = data_matrix[num, 15]
      output_matrix[icount, 5] = abs(round(data_matrix[num, 14], digits = 2))
      output_matrix[icount, 11] = 9999
      output_matrix[icount, 12] = 9999
      output_matrix[icount, 13] = 9999
      output_matrix[icount, 6] = 9999
      
      ####Event Type and Iteration####
      output_matrix[icount, 10] = event[1]
      icount = icount + 1
    }
  }
  
  # New conditional statement for duplicate dates with equal onset time and duration
  unique_dates = unique(output_matrix[, c(1, 2, 3)])
  for (i in 1:nrow(unique_dates)) {
    duplicate_rows = which(output_matrix[, 1] == unique_dates[i, 1] & output_matrix[, 2] == unique_dates[i, 2] & output_matrix[, 3] == unique_dates[i, 3])
    if (length(duplicate_rows) > 1) {
      output_matrix[duplicate_rows[1], 14] = days_of_year_table[which(days_of_year_table[, 1] == unique_dates[i, 1]), 3]
      output_matrix = output_matrix[-duplicate_rows[-1], ]
    }
  }
  
  large_table = data.frame("Year" = as.numeric(output_matrix[, 14]),
                           "Month" = as.numeric(output_matrix[, 1]),
                           "Day" = as.numeric(output_matrix[, 7]),
                           "Onset_time" = round(as.numeric(output_matrix[, 2]), digits = 2),
                           "Duration" = round(as.numeric(output_matrix[, 3]), digits = 2),
                           "Time_Max_Speed" = as.numeric(output_matrix[, 4]),
                           "Max_Speed" = as.numeric(output_matrix[, 5]),
                           "MJO_Phase" = as.numeric(output_matrix[, 8]),
                           "MJO_Mag" = as.numeric(output_matrix[, 9]),
                           "Event_Type" = output_matrix[, 10],
                           "Flip2_Onset_Time" = round(as.numeric(output_matrix[, 11]), digits = 2),
                           "Flip2_Duration" = round(as.numeric(output_matrix[, 12]), digits = 2),
                           "Flip2_Time_Max_speed" = as.numeric(output_matrix[, 13]),
                           "Flip2_Max_Speed" = as.numeric(output_matrix[, 6]))
  
  return(na.omit(large_table))
}

### Land/Sea Breeze Detection Function (Separation Function)

In [ ]:
land_sea_breeze_detection_algorithm = function(data_set, time_file, day_file_sea, day_file_land, sampling_frequency){
  z = seq(1, length(data_set), by = 1)
  timing = matrix(nrow = length(z), ncol = 5)
  total_data_matrix = matrix(nrow = length(z), ncol = 37)
  day_count_sea = -1
  day_count_land = -1
  num_count = 0
  num_count_land = 0
  second_flip_max = rep(NA, 3)
  second_flip_max_land = rep(NA, 3)
  first_event = 0
  end_function = FALSE
  for (num in z){
    if (end_function == TRUE){
      break
    }
    
    if (data_set[num] < 0 && data_set[num + 1] > 0 && data_set[num + (3 * (1 / sampling_frequency))] > 0){
      
      #Calculate the zero crossing from the index value#
      first_zero_crossing_index = zero_crossing(time_1 = z[num],
                                                value_t1 = data_set[num],
                                                time_2 = z[num+1],
                                                value_t2 = data_set[num+1])
      
      #Convert the zero crossing index to actual time#
      first_zero_crossing_time = convert_index_to_time(first_zero_crossing_index, time_file, sampling_frequency)
      
      #Check for an interruption#
      if (day_file_sea[num + 1] == day_count_sea){
        timing[num, 2] = first_zero_crossing_time
        z.=seq(z[num] + 1, z[num] + (25 * (1 / sampling_frequency)), by = 1)
        icount = 1
        duration_vector = rep(NA, (25 * (1 / sampling_frequency)))
        speed_matrix_2 = matrix(nrow = (25 * (1 / sampling_frequency)), ncol = 2)
        for (sea in z.){
          if (is.na(data_set[sea]) == TRUE){
            end_function = TRUE
            break
          }
          if (data_set[sea] > 0){
            duration_vector[icount] = time_file[sea]
            speed_matrix_2[icount, 1] = data_set[sea]
            speed_matrix_2[icount, 2] = time_file[sea]
            end_duration_calc_index = z[sea]
            icount = icount + 1
          } else {
            end_duration_calc_index2 = z[sea - 1]
            end_duration_calc_index_plus2 = z[sea]
            second_cross_values2 = matrix(nrow = 1, ncol = 1)
            second_cross_values2[1, 1] = data_set[sea]
            break
          }
        }
        
        length_speed_matrix2 = length(na.trim(speed_matrix_2[, 1]))
        second_zero_crossing_index2 = zero_crossing(time_1 = end_duration_calc_index2,
                                                    value_t1 = speed_matrix_2[length_speed_matrix2, 1],
                                                    time_2 = end_duration_calc_index_plus2,
                                                    value_t2 = second_cross_values2[1, 1])
        
        total_data_matrix[num, 7] = length(na.omit(duration_vector)) - 1 + (z[num + 1] - first_zero_crossing_index) + -1 *(end_duration_calc_index2 - second_zero_crossing_index2)
        second_flip_max[3] = length(na.omit(duration_vector)) -1 + (z[num + 1] - first_zero_crossing_index) + -1 *(end_duration_calc_index2 - second_zero_crossing_index2)
        total_data_matrix[num, 8] = max(na.omit(speed_matrix_2[, 1]))
        second_flip_max[1] = max(na.omit(speed_matrix_2[, 1]))
        
        speed_matrix_2[is.na(speed_matrix_2)] = -9999
        
        for (count in 1:(25 * (1 / sampling_frequency))){
          if (speed_matrix_2[count, 1] == max(na.omit(speed_matrix_2[, 1]))) {
            total_data_matrix[num, 11] = speed_matrix_2[count, 2]
            second_flip_max[2] = speed_matrix_2[count, 2]
          } else {
            next
          }
        }
        num_count = -1
        total_data_matrix[num, 26] = timing[num, 2]
        total_data_matrix[num, 29] = second_flip_max[3]
        total_data_matrix[num, 19] = space_vector[1]
        total_data_matrix[num, 31] = space_vector[4]
        total_data_matrix[num, 33] = space_vector[2]
        total_data_matrix[num, 34] = space_vector[3]
        
        if (second_flip_max[1] >= space_vector[2]){
          total_data_matrix[num, 21] = second_flip_max[2]
          total_data_matrix[num, 20] = second_flip_max[1]
        } else {
          total_data_matrix[num, 21] = space_vector[3]
          total_data_matrix[num, 20] = space_vector[2]
        }
      } else {
        if (num_count > 0){
          total_data_matrix[num_count, 13] = space_vector[1]
          total_data_matrix[num_count, 14] = space_vector[2]
          total_data_matrix[num_count, 15] = space_vector[3]
          total_data_matrix[num_count, 25] = space_vector[4]
        }
        timing[num, 1] = first_zero_crossing_time
        day_count_sea[1] = day_file_sea[num + 1]
        z.= seq(z[num] + 1, z[num] + (25 * (1 / sampling_frequency)), by = 1)
        icount = 1
        duration_vector = rep(NA, (25 * (1 / sampling_frequency)))
        speed_matrix = matrix(nrow = (25 * (1 / sampling_frequency)), ncol = 2)
        for (sea in z.){
          if (is.na(data_set[sea]) == TRUE){
            end_function = TRUE
            break
          }
          if (data_set[sea] > 0){
            duration_vector[icount] = time_file[sea]
            speed_matrix[icount, 1] = data_set[sea]
            speed_matrix[icount, 2] = time_file[sea]
            icount = icount + 1
          } else {
            end_duration_calc_index = z[sea - 1]
            end_duration_calc_index_plus = z[sea]
            second_cross_values = matrix(nrow = 1, ncol = 1)
            second_cross_values[1, 1] = data_set[sea]
            break
          }
        }
        
        length_speed_matrix = length(na.trim(speed_matrix[, 1]))
        second_zero_crossing_index = zero_crossing(time_1 = end_duration_calc_index,
                                                   value_t1 = speed_matrix[length_speed_matrix, 1],
                                                   time_2 = end_duration_calc_index_plus,
                                                   value_t2 = second_cross_values[1, 1])
        total_data_matrix[num, 1] = length(na.omit(duration_vector)) - 1 + (z[num + 1] - first_zero_crossing_index) + -1 *(end_duration_calc_index-second_zero_crossing_index)
        total_data_matrix[num, 2] = max(na.omit(speed_matrix[, 1]))
        
        speed_matrix[is.na(speed_matrix)] = -9999
        
        for (count in 1:(25 * (1 / sampling_frequency))){
          if (speed_matrix[count, 1] == max(speed_matrix[, 1])) {
            total_data_matrix[num, 5] = speed_matrix[count, 2]
          } else {
            next
          }
        }
        space_vector = rep(NA, 4)
        space_vector[1] = total_data_matrix[num, 1]
        space_vector[2] = total_data_matrix[num, 2]
        space_vector[3] = total_data_matrix[num, 5]
        space_vector[4] = timing[num, 1]
        num_count = z[num]
        
        if (first_event == 0){
          total_data_matrix[num_count, 13] = space_vector[1]
          total_data_matrix[num_count, 14] = space_vector[2]
          total_data_matrix[num_count, 15] = space_vector[3]
          total_data_matrix[num_count, 25] = space_vector[4]
          first_event = 12
        }
      }
    } else {
      if (data_set[num] > 0 && data_set[num + 1] < 0 && data_set[num + (3 * (1 / sampling_frequency))] < 0){
        
        #Calculate the zero crossing from the index value#
        first_zero_crossing_index_land= zero_crossing(time_1 = z[num],
                                                      value_t1 = data_set[num],
                                                      time_2 = z[num + 1],
                                                      value_t2 = data_set[num + 1])
        
        #Convert the zero crossing index to actual time#
        first_zero_crossing_time_land= convert_index_to_time(first_zero_crossing_index_land, time_file, sampling_frequency)
        
        #Check for an interruption event#
        if (day_file_land[num + 1] == day_count_land){
          timing[num, 4] = first_zero_crossing_time_land
          jcount = 1
          duration_vector = rep(NA, (25 * (1 / sampling_frequency)))
          speed_matrix = matrix(nrow = (25 * (1 / sampling_frequency)), ncol = 2)
          z.. = seq(z[num] + 1, z[num] + (25 * (1 / sampling_frequency)), by = 1)
          for (land in z..){
            if (is.na(data_set[land]) == TRUE){
              end_function = TRUE
              break
            }
            if (data_set[land] < 0){
              duration_vector[jcount] = time_file[land]
              speed_matrix[jcount, 1] = data_set[land]
              speed_matrix[jcount, 2] = time_file[land]
              jcount = jcount + 1
            } else {
              end_duration_calc_index_land = z[land - 1]
              end_duration_calc_index_plus_land = z[land]
              second_cross_values_land = matrix(nrow = 1, ncol = 1)
              second_cross_values_land[1, 1] = data_set[land]
              break
            }
          }
          
          length_speed_matrix_land = length(na.trim(speed_matrix[, 1]))
          second_zero_crossing_land_index = zero_crossing(time_1 = end_duration_calc_index_land,
                                                          value_t1 = speed_matrix[length_speed_matrix_land, 1],
                                                          time_2 = end_duration_calc_index_plus_land,
                                                          value_t2 = second_cross_values_land[1, 1])
          
          total_data_matrix[num, 9] = length(na.omit(duration_vector)) - 1 + (z[num + 1] - first_zero_crossing_index_land) + -1 *(end_duration_calc_index_land-second_zero_crossing_land_index)
          second_flip_max_land[3] = length(na.omit(duration_vector)) - 1 + (z[num + 1] - first_zero_crossing_index_land) + -1 *(end_duration_calc_index_land-second_zero_crossing_land_index)
          total_data_matrix[num, 10] = min(na.omit(speed_matrix[, 1]))
          second_flip_max_land[1] = min(na.omit(speed_matrix[, 1]))
          
          speed_matrix[is.na(speed_matrix)] = 9999
          
          for (count in 1:(25 * (1 / sampling_frequency))){
            if (speed_matrix[count, 1] == min(na.omit(speed_matrix[, 1]))){
              total_data_matrix[num, 12] = speed_matrix[count, 2]
              second_flip_max_land[2] = speed_matrix[count, 2]
            } else {
              next
            }
          }
          num_count_land = -1
          total_data_matrix[num, 28] = timing[num, 4]
          total_data_matrix[num, 30] = second_flip_max_land[3]
          total_data_matrix[num, 22] = space_vector_land[1]
          total_data_matrix[num, 32] = space_vector_land[4]
          total_data_matrix[num, 35] = space_vector_land[2]
          total_data_matrix[num, 36] = space_vector_land[3]
          if (second_flip_max_land[1] <= space_vector_land[2]){
            total_data_matrix[num, 24] = second_flip_max_land[2]
            total_data_matrix[num, 23] = second_flip_max_land[1]
          } else {
            total_data_matrix[num, 24] = space_vector_land[3]
            total_data_matrix[num, 23] = space_vector_land[2]
          }
        } else {
          if (num_count_land > 0){
            total_data_matrix[num_count_land, 16] = space_vector_land[1]
            total_data_matrix[num_count_land, 17] = space_vector_land[2]
            total_data_matrix[num_count_land, 18] = space_vector_land[3]
            total_data_matrix[num_count_land, 27] = space_vector_land[4]
          }
          timing[num, 3] = first_zero_crossing_time_land
          day_count_land[1] = day_file_land[num+1]
          jcount = 1
          duration_vector = rep(NA, (25 * (1 / sampling_frequency)))
          speed_matrix = matrix(nrow = (25 * (1 / sampling_frequency)), ncol = 2)
          z..= seq(z[num] + 1, z[num] + (25 * (1 / sampling_frequency)), by = 1)
          for (land in z..){
            if (is.na(data_set[land]) == TRUE){
              end_function = TRUE
              total_data_matrix[num_count, 13] = space_vector[1]
              total_data_matrix[num_count, 14] = space_vector[2]
              total_data_matrix[num_count, 15] = space_vector[3]
              total_data_matrix[num_count, 25] = space_vector[4]
              break
            }
            if (data_set[land] < 0){
              duration_vector[jcount] = time_file[land]
              speed_matrix[jcount, 1] = data_set[land]
              speed_matrix[jcount, 2] = time_file[land]
              jcount = jcount + 1
            } else {
              end_duration_calc_index_land  = z[land - 1]
              end_duration_calc_index_plus_land = z[land]
              second_cross_values_land = matrix(nrow = 1, ncol = 1)
              second_cross_values_land[1, 1] = data_set[land]
              break
            }
          }
          
          length_speed_matrix_land = length(na.trim(speed_matrix[, 1]))
          second_zero_crossing_land_index = zero_crossing(time_1 = end_duration_calc_index_land,
                                                          value_t1 = speed_matrix[length_speed_matrix_land, 1],
                                                          time_2 = end_duration_calc_index_plus_land,
                                                          value_t2 = second_cross_values_land[1, 1])
          total_data_matrix[num, 3] =length(na.omit(duration_vector)) - 1 + (z[num+1]-first_zero_crossing_index_land) + -1 *(end_duration_calc_index_land-second_zero_crossing_land_index)
          total_data_matrix[num, 4] = min(na.omit(speed_matrix[, 1]))
          
          
          speed_matrix[is.na(speed_matrix)] = 9999
          
          for (count in 1:(25 * (1 / sampling_frequency))){
            if (speed_matrix[count, 1] == min(na.omit(speed_matrix[,1]))){
              total_data_matrix[num, 6] = speed_matrix[count, 2]
            } else {
              next
            }
          }
          space_vector_land = rep(NA, 3)
          space_vector_land[1] = total_data_matrix[num, 3]
          space_vector_land[2] = total_data_matrix[num, 4]
          space_vector_land[3] = total_data_matrix[num, 6]
          space_vector_land[4] = timing[num, 3]
          num_count_land = z[num]
        }
      } else {
        timing[num, 5] = 9999
        total_data_matrix[num, 37] = 9999
      }
    }}
  return(total_data_matrix)
}

## Variable Creation

In [ ]:
#####Time Variables#####
num_days = calculate_num_days(data_start_date, data_end_date)
timefile = create_time_vector(local_zero_utc, num_days, sampling_rate)
days_of_year_table = create_days_of_year_table(data_start_date, data_end_date)

######Land Breeze Day Vector######
day_vector_land = generate_day_vector(local_zero_utc, land_breeze_window_start_time, sampling_rate, num_days)

######Sea Breeze Day Vector Definition######
day_vector_sea = generate_day_vector(local_zero_utc, sea_breeze_window_start_time, sampling_rate, num_days)

######MJO Data######
mjo_data = filter_mjo_data(MJO_RMM_file_location, data_start_date, data_end_date)

####Import Data####
setwd(data_file_path)

# List files based on the specified file type *
file_list = list.files(pattern = paste0("*.", file_type))

# Read contents of data file #
file = nc_open(file_list)
u_comp = ncvar_get(file, u_comp_name)
v_comp = ncvar_get(file, v_comp_name)
nc_long = ncvar_get(file, "longitude")
nc_lat = ncvar_get(file, "latitude")
nc_close(file)

# Find the indices of the closest latitude and longitude
lat_index = which.min(abs(nc_lat - lat_interest))
lon_index = which.min(abs(nc_long - lon_interest))

# Remove the data for the selected grid point
u.slice = u_comp[lon_index, lat_index, ]
v.slice = v_comp[lon_index, lat_index, ]

#####Convert from components to DD and FF#####
wind_speed = sqrt(u.slice^2 + v.slice^2)
wind_dir = 180 + (180 / pi) * atan2(u.slice, v.slice)

## Grid Rotation

In [ ]:
######Rotation Angle Determination#####
if (is.numeric(coordinate_axis_rotation_user)) {
  coordinate_axis_rotation = round(coordinate_axis_rotation_user, digits = 2)
  print(paste("The user-specified rotation angle is", coordinate_axis_rotation, "degrees."))
} else {
  coordinate_axis_rotation = round(cablculate_angle_from_x_axis(lat1, lon1, lat2, lon2, lat3, lon3), digits = 2)
  print(paste("The calculated rotation angle is", coordinate_axis_rotation, "degrees."))
}

#####Rotate the Axis and Calculate the u and v components#####
u.rot_obs = -wind_speed * sin((pi / 180) * (wind_dir + coordinate_axis_rotation))
v.rot_obs = -wind_speed * cos((pi / 180) * (wind_dir + coordinate_axis_rotation))

## Filtering Algorithm

In [ ]:
#####Apply the 3-day Chebyshev Type I#####
low_pass_filter_results = pass.filt(u.rot_obs, W = (1 / (synoptic_filter_period * (24 * (1 / sampling_rate)))),type = "low",method = "ChebyshevI")

#####Subtract the Filter from Shore-Perpendicular Wind#####
no_synoptic_winds = u.rot_obs-low_pass_filter_results
filtered_winds_final = pass.filt(no_synoptic_winds, W = (1 / (noise_filter_period * (1 / sampling_rate))), type = "low", method = "ChebyshevI")

#####Run the Function#####
results_filtering_algorithm = land_sea_breeze_detection_algorithm(filtered_winds_final, timefile, day_vector_sea, day_vector_land, sampling_rate)

## Data Table Output

In [ ]:
#####Table Creation#####
output_data_table_land = table_maker_land(results_filtering_algorithm, local_zero_utc, land_breeze_window_start_time)
output_data_table_sea = table_maker_sea(results_filtering_algorithm,local_zero_utc, sea_breeze_window_start_time)

#####Table Export######
setwd(data_table_output_directory)
write.csv(output_data_table_land, file = output_file_name_land, row.names = T)
write.csv(output_data_table_sea, file = output_file_name_sea, row.names = T)